In [1]:
import cv2

import mediapipe as mp

In [ ]:
import numpy as np
def calculate_angle(a, b, c):
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
    return angle

In [ ]:
import cv2
import mediapipe as mp
import numpy as np


mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Fungsi untuk menghitung sudut
def calculate_angle(a, b, c):
    """Menghitung sudut antara tiga titik dalam koordinat (x, y)."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Hindari error numerik
    return np.degrees(angle)

def fall_detector(video_path):
    """Mendeteksi jatuh menggunakan MediaPipe Pose dari video MP4."""
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, 
                         min_detection_confidence=0.5, min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path) 
    if not cap.isOpened():
        print("Gagal membuka video. Periksa path file!")
        return

    fall_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            print("Landmark berhasil terdeteksi!")  
            landmarks = results.pose_landmarks.landmark

            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                                      mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))


            nose = [landmarks[mp_pose.PoseLandmark.NOSE].x * frame.shape[1], 
                    landmarks[mp_pose.PoseLandmark.NOSE].y * frame.shape[0]]

            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * frame.shape[1], 
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * frame.shape[0]]

            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * frame.shape[1], 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * frame.shape[0]]

            mid_hip = [(left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2]

            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * frame.shape[1], 
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * frame.shape[0]]

            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * frame.shape[1], 
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * frame.shape[0]]

            mid_knee = [(left_knee[0] + right_knee[0]) / 2, (left_knee[1] + right_knee[1]) / 2]

            angle = calculate_angle(nose, mid_hip, mid_knee)
            print(f"Sudut torso: {angle:.2f}°") 

            status = "Tidak Jatuh"
            color = (0, 255, 0)  # Hijau jika tidak jatuh
            if angle < 30 and mid_hip[1] > 0.8 * frame.shape[0]:  # Mid-hip turun ke bawah
                status = "Jatuh"
                color = (0, 0, 255)  # Merah jika jatuh
                if not fall_detected:
                    print("⚠️ Jatuh terdeteksi!")
                    fall_detected = True
            else:
                fall_detected = False

            min_x = int(min(nose[0], left_hip[0], right_hip[0], left_knee[0], right_knee[0])) - 20
            max_x = int(max(nose[0], left_hip[0], right_hip[0], left_knee[0], right_knee[0])) + 20
            min_y = int(min(nose[1], left_hip[1], right_hip[1], left_knee[1], right_knee[1])) - 20
            max_y = int(max(nose[1], left_hip[1], right_hip[1], left_knee[1], right_knee[1])) + 20

            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), color, 3)

            cv2.putText(frame, status, (min_x, min_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, color, 2, cv2.LINE_AA)

        else:
            print("⚠️ Landmark tidak terdeteksi!")  # Debugging

        cv2.imshow('Fall Detection', frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):  # Delay agar sesuai FPS video
            break

    cap.release()
    cv2.destroyAllWindows()

fall_detector('sample_input_2.mp4')  # Ganti dengan nama file video yang ada


Landmark berhasil terdeteksi!
Sudut torso: 157.79°
Landmark berhasil terdeteksi!
Sudut torso: 159.07°
Landmark berhasil terdeteksi!
Sudut torso: 160.26°
Landmark berhasil terdeteksi!
Sudut torso: 161.83°
Landmark berhasil terdeteksi!
Sudut torso: 160.43°
Landmark berhasil terdeteksi!
Sudut torso: 160.26°
Landmark berhasil terdeteksi!
Sudut torso: 160.36°
Landmark berhasil terdeteksi!
Sudut torso: 160.74°
Landmark berhasil terdeteksi!
Sudut torso: 159.94°
Landmark berhasil terdeteksi!
Sudut torso: 159.06°
Landmark berhasil terdeteksi!
Sudut torso: 162.76°
Landmark berhasil terdeteksi!
Sudut torso: 162.78°
Landmark berhasil terdeteksi!
Sudut torso: 159.83°
Landmark berhasil terdeteksi!
Sudut torso: 159.13°
Landmark berhasil terdeteksi!
Sudut torso: 161.60°
Landmark berhasil terdeteksi!
Sudut torso: 160.59°
Landmark berhasil terdeteksi!
Sudut torso: 161.23°
Landmark berhasil terdeteksi!
Sudut torso: 157.07°
Landmark berhasil terdeteksi!
Sudut torso: 159.44°
Landmark berhasil terdeteksi!
S

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import os
import requests
from datetime import datetime

class FallDetector:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Parameter deteksi jatuh
        self.fall_threshold_angle = 60
        self.hip_height_threshold = 0.3
        
        # Telegram settings
        self.TOKEN = "7393109138:AAH4t1OfWQwYXQJihAuhrEEiYTf_DKGILds"
        self.CHAT_ID = "-1002491643130"
        
        # Output directory
        self.output_dir = "fall_detected_images"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            
        self.previous_fall_status = False

    def send_telegram_message(self, message):
        url = f"https://api.telegram.org/bot{self.TOKEN}/sendMessage"
        payload = {
            "chat_id": self.CHAT_ID,
            "text": message
        }
        requests.post(url, json=payload)

    def send_telegram_photo(self, photo_path, caption):
        url = f"https://api.telegram.org/bot{self.TOKEN}/sendPhoto"
        with open(photo_path, 'rb') as photo:
            files = {'photo': photo}
            data = {
                'chat_id': self.CHAT_ID,
                'caption': caption
            }
            requests.post(url, data=data, files=files)
        
    def calculate_angle(self, a, b, c):
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        return np.degrees(angle)
    
    def save_fall_image(self, frame):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        filename = f"fall_detected_{timestamp.replace(':', '-')}.jpg"
        filepath = os.path.join(self.output_dir, filename)
        cv2.imwrite(filepath, frame)
        
        # Kirim notifikasi dan gambar ke Telegram
        self.send_telegram_message(f"⚠️ ALERT! Fall Detected at {timestamp}!")
        self.send_telegram_photo(filepath, f"Fall Detected at {timestamp}")
        
        print(f"Saved and sent fall detection image: {filepath}")

    def detect_falls(self, video_path):
        cap = cv2.VideoCapture(video_path)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.pose.process(rgb_frame)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                self.mp_drawing.draw_landmarks(
                    frame, 
                    results.pose_landmarks,
                    self.mp_pose.POSE_CONNECTIONS
                )

                shoulder = [landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1],
                            landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]]
                hip = [landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x * frame.shape[1],
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y * frame.shape[0]]
                knee = [landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x * frame.shape[1],
                        landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y * frame.shape[0]]

                body_angle = self.calculate_angle(shoulder, hip, knee)
                hip_height_ratio = landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y

                is_falling = (body_angle < self.fall_threshold_angle and 
                            hip_height_ratio > self.hip_height_threshold)

                if is_falling and not self.previous_fall_status:
                    self.save_fall_image(frame)
                
                self.previous_fall_status = is_falling

                status = "FALL DETECTED!" if is_falling else "Normal"
                color = (0, 0, 255) if is_falling else (0, 255, 0)
                
                cv2.putText(frame, f"Status: {status}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                cv2.putText(frame, f"Body Angle: {body_angle:.1f}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            cv2.imshow('Fall Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# Penggunaan
detector = FallDetector()
detector.detect_falls('sample_4.mp4')


Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-00.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-08.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-13.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-19.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-25.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-31.jpg
Saved and sent fall detection image: fall_detected_images\fall_detected_2025-02-18 19-36-37.jpg
